<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

# TensorFlowによる画像セグメンテーション入門

多くの重要な画像解析タスクに対するディープラーニングの応用では、ある画像にみられる個々のオブジェクトを認識することを越えて、関心のある空間領域に画像を分割する必要があります。例えば医用画像解析では、異なるタイプの組織、血液、あるいは異常細胞に対応するピクセルを分離して、特定の器官を分離することが重要です。このハンズオンラボでは、 [TensorFlow](https://www.tensorflow.org) の機械学習フレームワークにより、医用画像データセットを用いて画像セグメンテーションネットワークを学習及び評価します。

このラボは Jonathan Bentz(Twitter:[@jnbntz](https://twitter.com/jnbntz))によって作成されました。

---
ラボを始める前に、 [WebSockets](http://en.wikipedia.org/wiki/WebSocket) があなたのシステムで動作していることを確認しましょう。これを行うには、下にあるセルブロックをクリックし、Ctrl + Enterを押すか、上のツールバーの再生ボタンを押して実行します。問題がなければ、灰色のセルの下に出力が返されるのが確認できるはずです。できない場合は、[Self-paced Lab Troubleshooting FAQ](https://developer.nvidia.com/self-paced-labs-faq#Troubleshooting) を参考に、問題をデバッグしてください。

In [ ]:
print "The answer should be three: " + str(1+2)

下のセルを実行し、サーバー上で動作しているGPUに関する情報を表示しましょう。

In [ ]:
!nvidia-smi

さらに、下のセルを実行して、このラボで使用するTensorflowのバージョンを表示します。

In [ ]:
!python -c 'import tensorflow as tf; print(tf.__version__)'

NVIDIAが提供する、IPython Notebookベースの自習型ラボをまだ使ったことのない方は、YouTubeにて[こちら](http://www.youtube.com/embed/ZMrDaLSFqpY)のビデオをチェックすることをおすすめします。

# 画像セグメンテーション

このラボでは、セマンティックセグメンテーションとも呼ばれる、画像セグメンテーションの演習を行います。
セマンティックセグメンテーションとは、各ピクセルを特定のクラスに分類するタスクのことです。
ある意味では、画像全体ではなくピクセル単位での分類問題とみなすこともできます。
このラボでは、心臓MRI画像における各ピクセルが、左心室（LV）に属するかどうかを分類することが課題となります。

ここでは、ディープラーニングの入門的な解説は行いませんし、畳み込みネットワークの数式を厳密に追いかけることもしません。
少なくとも、誤差逆伝播法や活性化関数、確率的勾配降下法（SGD）、畳み込み、プーリング、バイアスなど、ニューラルネットワークに関して一通り理解していることを前提とします。
既に畳み込みネットワークに触れたことがあり、画像認識タスクについて理解していると尚良いでしょう。
このラボでは、Googleの機械学習フレームワークであるTensorFlowを使用するので、必須ではありませんがPythonとTensorFlowの経験があると有利でしょう。　ここではコーディングそのものよりも、TensorFlowを用いて学習や評価タスクの設定を行うことがほとんどになります。

## 入力データセット

使用するデータセットは、専門家によってラベルが付けられた連続心臓画像（MRI短軸（SAX）断面図）です。引用元についての詳細情報は、参考文献 [[1](#1), [2](#2), [3](#3)] を参照してください。

データセットの４つの代表例を以下に示します。
画像の各行はデータのインスタンスです。
左の列にはMRI画像が、右の列には専門家によってセグメントされた領域（輪郭とも呼ばれます）が表示されています。
左心室に該当する部分が白く塗られています。
左心室の大きさは画像によって異なりますが、一般的には画像全体の比較的小さな領域を占めます。

***
![Figure 1](./fig1_final.png)
***
![Figure 2](./fig2_final.png)
***
![Figure 3](./fig3_final.png)
***
![Figure 4](./fig4_final.png)
***

データ準備が機械学習ワークフローにおいて重要ですが、このラボの範囲を超えています。
そのため、画像の前処理や、TensorFlowに読み込ませるための処理については、このラボでは扱いません。

その詳細について興味のある方のために、以前開かれた、画像の適切な取得方法に関する[Kaggleコンペティション](https://www.kaggle.com/c/second-annual-data-science-bowl/details/deep-learning-tutorial)のガイダンスと、部分的なコードを入手しました。
画像を取得した時点で、それらをTensorFlowレコード（TFRecords）に変換してファイルに保存してあります。
[TFRecords](https://www.tensorflow.org/versions/r0.12/how_tos/reading_data/index.html)はTensorFlowによって提供される特殊なファイル形式で、TensorFlowのデータ管理用関数に渡すことができます。 こうした組み込み関数には、マルチスレッドでのデータ読み込みを行うものや、ランダム化やオーグメンテーションのように高度なデータの前処理を行うものなどがあります。

画像そのものは、医用画像で一般的に使われる、256×256グレースケール[DICOM](https://en.wikipedia.org/wiki/DICOM)形式です。 ラベルは256×256×2のテンソルです。
最後の次元が2なのは、ピクセルを2クラスのうち1つに分類できるように、サイズ2のベクトルを割り当てているからです。
訓練データは234枚の画像で、検証データ（学習には使用せず、モデルの精度をテストするために用いるデータ）は26枚です。

# TensorFlowによるディープラーニング

このラボは、現在パブリックに利用可能なディープラーニングフレームワークを自分のペースで体験するために作られたラボの一部です。 
 TensorFlowは[Google](https://www.google.com)によって開発されたフレームワークで、Googleの多数の研究者やプロダクトグループで使用されています。

TensorFlowは、オープンソースの機械学習ライブラリです。
ここでの計算は、名前の由来ともなっているテンソル上で動作するデータフローグラフで表現されます。
この方法で表現できるのであれば、任意のアルゴリズムをTensorFlowのフレームワークで実行することができます。

TensorFlowは、CPUでもGPUでも動かすことができ、サーバーやワークステーションでも利用できます。また、モデルをモバイルプラットフォーム上にデプロイすることもできるため、移植性が高いといえます。
現時点では、TensorFlowで計算を表現するのにPythonかC++が利用できるようになっており、実験的に[Go言語とJAVA](https://www.tensorflow.org/api_docs/)もサポートされています。
実際には、Pythonで学習とテストを行い、モデルが完成したらC++でデプロイするのが一般的です。

TensorFlowは、CPUとGPUの両方で実行できるように設計されています。
単一に実行する場合には、必要に応じてCPUとGPUに異なるタスクを明示的に割り当てることができるという点で、柔軟性があります。
GPUで実行する場合は、[cuDNN](https://developer.nvidia.com/cudnn)を含むいくつかのGPUライブラリを使用することにより、利用可能な最新のGPUから最大のパフォーマンスを引き出すことができます。

このラボの目的の1つは、TensorFlowを入門レベルで使いこなすことです。
この短時間のラボでは、TensorFlowのすべての機能やオプションについて議論することはできません。しかしこのラボを完了すれば、特定の機械学習の問題をTensorFlowを使ってどのように解決していけばいいか、落ち着いて考えることができるようになるでしょう。

TensorFlowの包括的なドキュメントについては、 [TensorFlow website](https://www.tensorflow.org)や [whitepaper](http://download.tensorflow.org/paper/whitepaper2015.pdf)、[GitHub site](https://github.com/tensorflow/tensorflow)を参照することをおすすめします。

# TensorFlowの基礎

TensorFlowは、好みに応じてさまざまな使い方をすることができます。
学習タスクを設計する際は、TensorFlow Python APIを利用するのが一般的です。
TensorFlowで機械学習のタスクを実行するには、少なくとも２つのステップを踏む必要があります。

## データフローグラフ

まず、データフローグラフを構築します。これは、あなたの行いたい計算を明確に順序付ける仕様書の役割を果たします。
TensorFlowのAPIを使用することで、畳み込みやアクティベーション、プーリングなど、TensorFlowが提供する演算を用いたニューラルネットワークを、レイヤーごとに構築することができます。
この段階では、まだ実際のデータを用いた計算は行いません。ただ指定したグラフを構築するだけです。

グラフを構築する際には、TensorFlow用語でいうところの`Variable`を指定する必要があります。
データを`Variable`として指定することで、それが"学習"されるパラメータ、すなわち、学習が進行するにつれて更新される重みであることをTensorFlowに伝えます。

## セッション

ニューラルネットワークをデータフローグラフとして定義したら、 `Session`を開始します。 これは、入力データと学習パラメータを、事前に構築されたグラフに提示して計算を進める仕組みです。

一般に、これらの２つのステップは、グラフを変更するたびに繰り返されます。つまり、グラフを更新したら新しいセッションを開始することになります。

# Sample Workflow

モデルの学習と評価のサンプルワークフローは次の通りです。

1. 入力データの準備--入力データはNumpy配列で与えられますが、TensorFlowは巨大なデータセット用にTFRecordsと呼ばれる形式も提供しています。
2. 計算グラフの構築--推論ノードや誤差ノード、学習ノードなど、特殊なノードを含むニューラルネットワークのグラフを作成します。
3. モデルの学習--入力データをTensorFlowの`Session`のグラフに流し込み、ループさせます。バッチサイズやepoch数、学習率などのカスタマイズも行います。
4. モデルの評価--学習の時と同じグラフを用いて未知のデータに対する推論を行い、適切な指標に基づいてモデルの精度を評価します。

# TensorBoard

TensorFlowには、プログラムを様々な側面から視覚化するための、[TensorBoard](https://www.tensorflow.org/get_started/summaries_and_tensorboard)とよばれる機能豊富なツールが用意されています。
TensorBoardでは、計算グラフを可視化し、誤差や精度、学習率などのさまざまな指標をプロットすることができます。
基本的に、TensorFlowの実行中に生成されたデータは、追加的なAPIをいくつか呼び出すことでTensorBoardに表示することができます。
たとえば次のような、一つの隠れ層を持つニューラルネットを作成するコードがあるとします。（ここではコードの詳細は気にしないでください）

```
with tf.name_scope('Hidden1'):
    W_fc = tf.Variable(tf.truncated_normal( [256*256, 512],
                 stddev=0.1, dtype=tf.float32), name='W_fc')
    flatten1_op = tf.reshape( images_re, [-1, 256*256])
    h_fc1 = tf.matmul( flatten1_op, W_fc )

with tf.name_scope('Final'):
    W_fc2 = tf.Variable(tf.truncated_normal( [512, 256*256*2],
                stddev=0.1, dtype=tf.float32), name='W_fc2' )
    h_fc2 = tf.matmul( h_fc1, W_fc2 )
    h_fc2_re = tf.reshape( h_fc2, [-1, 256, 256, 2] )

return h_fc2_re
```

TensorBoardは以下の図のようなニューラルネットワークを表示します。
よく見てみると、エッジの部分にはテンソルの次元が書かれていることわかります。つまり、ノードからノードへと辿っていくことで、テンソルとしてのデータとそのサイズの変化を追うことができるのです。

![TensorBoard Example](./hidden1.png)

# 課題1 -- １つの隠れ層

![NN](./NN.png)

私たちが取り組む最初の課題は、1つの隠れ層をもつ全結合ニューラルネットワークを作成し、学習し、テストすることです。
ニューラルネットワークへの入力は、各ピクセルの値、すなわち256×256（あるいは65,536）の配列です。
隠れ層のサイズは任意で、出力は256×256×2の配列となります。つまり、各入力ピクセルについて、2つのクラスに属する確率がそれぞれ出力されます。
今回の場合は、左心室であるか否かが2つのクラスとなります。
誤差の計算は、[`sparse_softmax_cross_entropy_with_logits`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#sparse_softmax_cross_entropy_with_logits)と呼ばれる関数を用いて行います。これは、ソフトマックスとクロスエントロピーの計算を一つの関数にまとめたものです。

## 学習

最初の練習では既にソースコードが用意されています。
ニューラルネットの学習を始めるには、以下のセルを実行します。

In [ ]:
!python exercises/simple/runTraining.py --data_dir /data

正しく動作していれば、画面にいくつかのメッセージが表示されます。
このうちのいくつかはTensorFlowが出力するメッセージです。通常は無視してかまいません。
"OUTPUT"で始まる行を探してください。これは、100ステップ毎にその時点での誤差などを出力するように我々がプログラムしておいたため表示されているものです。
最後の行は次のようになっています。

`OUTPUT: Done training for 1 epochs, 231 steps`.  

これは、全てのデータに対して学習が一巡し、1epochが完了したことを意味します。

# 評価

学習済みモデルが得られたら、そのモデルにとって未知のデータに対してどれくらい機能するかを評価したいところです。
学習済みモデルの評価を行うには、以下のセルを実行します。

In [ ]:
!python exercises/simple/runEval.py --data_dir /data

再び、TensorFlowの出力の大半は無視して、"OUTPUT"で始まる行に注目しましょう。
実行結果は以下のようになりました。あなたの結果も同じようになっているはずです。

```
OUTPUT: 2017-01-26 17:12:28.929741: precision = 0.503
OUTPUT: 26 images evaluated from file /data/val_images.tfrecords
```

最後の方の行には、モデルの精度が表示されています。すなわち、ground truth（正解ラベル）と照らし合わせた結果、各ピクセルが左心室であるか否かをどれだけうまく予測できているかを表しています。
上のケースでは、0.503すなわち50.3%の精度が得られており、おおよそ半分のピクセルを正確に分類できたことになります。これはあまり良い結果ではないかもしれませんが、シンプルなネットワークを1epoch学習させただけだと考えると、それほど悪くはないのではないでしょうか。

## TensorBoard

この時点でまだ開いていない場合、以下のリンクからTensorBoardを開きましょう。

### [Open TensorBoard](/tensorboard/)

TensorBoardには優れた可視化機能があります。
トップメニューの"Scalars"をクリックすると、キャプチャされた情報が表示されます。
それらをクリックして、データのプロットを確認することができます。

メニューの"Graphs"を選択すると、学習または評価のデータフローグラフを見ることができます。
グラフのノードをクリックすることで、そのノードに関する詳しい情報が得られます。
ページの左上にあるドロップダウンリストで、学習と評価のグラフを切り替えられます。

このタスクで使ったネットワークは、以下のコードで記述されています。

```
with tf.name_scope('Hidden1'):
    W_fc = tf.Variable(tf.truncated_normal( [256*256, 512],
                 stddev=0.1, dtype=tf.float32), name='W_fc')
    flatten1_op = tf.reshape( images_re, [-1, 256*256])
    h_fc1 = tf.matmul( flatten1_op, W_fc )

with tf.name_scope('Final'):
    W_fc2 = tf.Variable(tf.truncated_normal( [512, 256*256*2],
                stddev=0.1, dtype=tf.float32), name='W_fc2' )
    h_fc2 = tf.matmul( h_fc1, W_fc2 )
    h_fc2_re = tf.reshape( h_fc2, [-1, 256, 256, 2] )

return h_fc2_re

```

上のコードを見ると、PythonでTensorFlowのAPIを呼び出していることがわかるでしょう。
* `tf.name_scope()` を用いて、プログラムの特定のスコープに名前を付けます。コードを整理したいときや、TensorBoardで表示されるグラフのノードに名前を付けたいときに便利です。
* `tf.Variable()` は学習されるVariable、すなわち、重みのテンソルを指定します。
* `tf.reshape()` は、テンソルを後に続く演算に適した形状に変換するための、補助的な関数です。
* `tf.matmul()` はその名の通り、2つのテンソルの行列積を計算します。

### 詳細に触れなかったトピックについて

ここまでで触れていないトピックがいくつかあります。詳細な議論は行いませんが、一応補足しておきます。
* データは全て用意されていましたが、ここでは先述したTFRecordsというファイルデータ形式が使われています。 
* 私たちは、マルチスレッドでデータを読み込むためのTensorFlowの仕組みを利用しています。ここでは、データをランダム化するためのTensorFlowの組み込み関数を使ったり、`batch_size`や`num_epoch`といったものを自動的に扱ったりすることができます。
* 実際のモデル構造について、データフローグラフを通して簡単な説明を行ってきました。そのほとんどはPythonの構文で書かれているので、必要であればソースコードを確認してください。
* 最後に、TensorBoardにデータを転送し、簡単に可視化するためのAPIを呼び出すコードを追加しました。これについても、必要に応じてソースコードを確認してください。

# 課題2 -- 畳み込みニューラルネットワーク (CNN)

2つ目の課題は、先ほど作ったモデルにさらなる層や様式を加えることで、より洗練されたネットワークを作ることです。
前の例では個々のピクセルに注目しましたが、関心領域が1つのピクセルより大きいかもしれないということを考慮しませんでした。
今回はピクセルだけでなく、小さな領域も捉えて利用したいので、より大きな受容野を持つ畳み込み層を使います。

さらに、大部分の情報を保持しつつデータのダウンサンプリングを行うことが可能な、プーリング層を追加します。
これにより、計算の複雑さがいくらか解消されます。

ここまでは、画像認識ニューラルネットワークに関連して、出力ノードの数がクラスの数になるような層を扱ってきました。
しかし今回は、画像分類以上のことを行っているのだということを思い出してください。私たちが分類するのは各ピクセルなので、出力のサイズはクラス数（2）×ピクセル数（256×256）になるようにします。
さらに、出力ノードの空間的な位置も重要です。各ピクセルは、自身が左心室に含まれる（含まれない）確率を持つことになるからです。

CNNは、画像認識や画像分類のタスクに対応する優れた選択肢として確立しています。
このラボでの課題はセグメンテーションですが、ある意味で分類タスクに関連しています。
画像全体を分類するというより、画像の各ピクセルを分類するからです。
ここで浮かんでくる疑問は、画像認識ですでに成果を上げているものと同じタイプのCNNが、セグメンテーションにも利用できるのかということです。
それは、CNNにいくつかの変更を加えることによって、可能になります。

具体的には、標準的な画像認識ニューラルネットワークにおける全結合層（通常は最後のいくつかの層）を、逆畳み込み層（[転置畳み込み層](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#conv2d_transpose)と呼ぶ方が正確かもしれません）に置き換えます。

逆畳み込みは、ダウンサンプリングによって小さくなった画像データを、最終的な分類のために元のサイズに戻すアップサンプリング手法です。
この手法に関する資料としては、[[4](#4), [5](#5), [6](#6)]をお勧めします。
CNNをセグメンテーション用に変更して得られたネットワークは、一般にFully Convolutional Network（FCN）と呼ばれます。

入力データ（この場合は256×256×1のテンソル）がグラフをどのように"流れるか"、つまり、コンボリューション、プーリングなどのさまざまな演算を通してデータがどのように変換されるかを可視化しておくと便利です。 以下の図は、次のタスクでデータに施される変換を表しています。

![FCN](./FCN.png)

上図のネットワークは、[[7](#7)]にみられるネットワークと類似しています。
畳み込み層、プーリング層、逆畳み込み層で構成されており、入力画像は図のように変換されていきます。

課題は、このニューラルネットワークを完成させて学習を行うことです。
これを達成するために、[`exercises/cnn/neuralnetwork.py`](/NVIDIA13/edit/exercises/cnn/neuralnetwork.py)というファイルを編集し、`FIXME`と書かれている部分をコードに書き換えます。
ファイルにはあなたの助けになるようなコメントが挿入されています、また、以下のネットワーク構造も参考にしてください。
層の名称は、コードを書く際に意味を持ちます。

1. Convolution1, 5 x 5 kernel, stride 2
2. Maxpooling1, 2 x 2 window, stride 2
3. Convolution2, 5 x 5 kernel, stride 2
4. Maxpooling2, 2 x 2 window, stride 2
5. Convolution3, 3 x 3 kernel, stride 1
6. Convolution4, 3 x 3 kernel, stride 1
7. Score_classes, 1x1 kernel, stride 1
8. Upscore (deconvolution), 31 x 31 kernel, stride 16

自分の解答の正誤をチェックしたいときは、[`exercise_solutions/cnn/neuralnetwork.py`](/NVIDIA13/edit/exercise_solutions/cnn/neuralnetwork.py)を参照してください。

コードが完成したら、下のセルを実行して学習を開始し、前の課題で開いておいたTensorBoardで結果を可視化しましょう。
結果がすぐに表示されない場合は、少し待ちましょう。ブラウザの更新が必要な場合もあります。

In [ ]:
!python exercises/cnn/runTraining.py --data_dir /data

学習が完了したら、以下のセルを実行してモデルの精度を確かめましょう。

In [ ]:
!python exercises/cnn/runEval.py --data_dir /data

2つ上のセルで実行した`runTraining.py`では、いくつかのコマンドライン引数を渡すことにより、さまざまな学習パラメータを試すことができます。
もし時間があれば、`---num_epochs`を変更して実験し、epoch数が学習にどのような影響を与えるか確かめてみましょう。

以下は、利用可能なコマンドライン引数の一覧です。

```
optional arguments:
  -h, --help           
                        このヘルプメッセージを表示して終了
  --learning_rate LEARNING_RATE
                        学習率の初期値
  --decay_rate DECAY_RATE
                        学習率の減衰率
  --decay_steps DECAY_STEPS
                        学習率減衰のインターバル
  --num_epochs NUM_EPOCHS
                       　エポック数
  --data_dir DATA_DIR   
                       　学習データのあるディレクトリ
  --checkpoint_dir CHECKPOINT_DIR
                        モデルを保存するディレクトリ
```

注: ソースコードをよく見ると、バッチサイズを変更するオプションが見つかると思いますが、このラボではデフォルトの1のままにしておいてください。

最高でどこまでの精度が得られるでしょうか？
一つの例として、1epochの学習で56.7%に達しています。

```
OUTPUT: 2017-01-27 17:41:52.015709: precision = 0.567
```

epochを30まで増やすと、さらに精度は向上します。

```
OUTPUT: 2017-01-27 17:47:59.604529: precision = 0.983
```

epoch数を増やしたことで、精度が大幅に上昇することが分かりました。実際、98.3%という数字はかなり良いです。これで十分でしょうか？課題はこれで終わってしまっていいのでしょうか？

# 精度

精度について議論するためには、一歩引いて、正確には私たちが何を計算しているのかを考慮する必要があります。
私たちが今使っている精度の指標は、どれだけのピクセルを正確に分類できたかを教えてくれています。
したがって、上の30epochの場合だと、ピクセルの値が98.3%正しく計算されているということになります。
しかし、初めの方で確認した画像からわかるように、一般に左心室の領域は画像全体に比べてかなり小さくなっています。
これは、クラス不均衡と呼ばれる問題を引き起こします。すなわち、あるクラスに分類される確率が、他のクラスに分類される確率よりもはるかに高くなります。
今回の場合、全てのピクセルを左心室でないクラスに割り当てるようなネットワークを設計すれば、95%の精度が得られることになるでしょう。
しかしそんなネットワークが役に立たないのは明らかです。
私たちに必要なのは、不均衡に関係なくどれだけ上手く左心室をセグメントできたかを示してくれるような指標です。

# 課題 3 -- Dice Metricによる評価

ネットワークがどれほど上手く左心室をセグメントしているかを、より正確に判断するために使用できる指標の１つは、Dice metricあるいはSorensen-Dice係数と呼ばれるものです。
これは、2つのサンプルの類似性を比較するための指標です。
私たちの場合は、2つの関心領域を比較するためにこれを使います。すなわち、専門的にラベル付けされた領域と、予測された領域を比較します。
Dice metricの計算式は以下の通りです。

$$ \frac{2A_{nl}}{A_{n} + A_{l}} $$

ここで、$A_n$はニューラルネットワークが予測した領域、$A_l$は専門家によってラベル付けされた領域、$A_{nl}$は両者が重なった部分、すなわち、ネットワークが正しく予測できた領域です。1.0という値は精度が完璧であることを意味します。

この指標はクラス不均衡の問題を無効にしてくれるので、より正確にネットワークのパフォーマンスを評価してくれるでしょう。特定の領域にどのくらいの領域が含まれているかを判断しようとしているので、我々はピクセルを数えて領域を与えるだけで済みます。

Dice metricがどのように実装されているのか知りたい場合は、[`neuralnetwork.py`](/NVIDIA13/edit/exercises/cnnDice/neuralnetwork.py)のソースコードを確認してください。

以下のセルを実行して、1epochでの学習を始めましょう。さらにもう一つ下のセルも実行して、モデルの精度を評価しましょう。
次に、30epochでの学習も試してみましょう。おそらく前の課題のときと同じような結果になるでしょう。
TensorBoardでの可視化も行ってください。

In [ ]:
!python exercises/cnnDice/runTraining.py --data_dir /data --num_epochs 1

In [ ]:
!python exercises/cnnDice/runEval.py --data_dir /data

1epochの場合、1%にも達しないかもしれません。我々が1epochで実行した際には、以下の結果が得られました。

```
OUTPUT: 2017-01-27 18:44:04.103153: Dice metric = 0.034
```

30epochで試してみれば、57%くらいになるでしょう。

```
OUTPUT: 2017-01-27 18:56:45.501209: Dice metric = 0.568
```

現実的な指標を用いたことで、ニューラルネットワークにはまだ改善の余地があるということが分かったのではないでしょうか。

# パラメータ探索

ここまで、適切な構造を持っていると思われるネットワークを作成し、セグメンテーションが上手くいっているかどうか適切に評価できる精度指標を使用してきました。
しかし、現時点での精度はそれほど高くありませんでした。
次に行うべきは、パラメータ空間をもう少し探索してみることです。今まで調整してきたパラメータはepoch数だけでしたが、他にも試せるパラメータはいくつかあります。それらも精度の向上に関わるかもしれません。具体的には、以下のようなものがあります。

* --learning_rate: 学習率の初期値
* --decay_rate: 初期の学習率を減衰させる割合、 たとえば、1.0 は減衰なしを、 0.5は元の半分ずつ減衰していくことを意味します。
* --decay_steps: 学習率を減衰させる際のインターバル

学習率は、バックプロパゲーションで毎回重みを更新する際の調整度合いです。学習率が大きすぎると、重みが大きく変更されてしまうため、理想的な解に上手くおさまることができません。一方、学習率が小さすぎると、重みがほとんど変更されないため、理想的な解にたどり着くまで非常に時間がかかってしまいます。
よく用いられる方法の一つとして、学習率を可変とすることがあげられます。
学習の始めには、大雑把な重みの調整によって理想的な解へ近づくことを期待して、学習率を大きくしておきます。
そして学習を進めつつ、解へと照準を合わせるべく、学習率を連続的に下げていきます。
先述した3つのパラメータによって、学習率と変更の程度、変更頻度をコントロールします。
これらのオプションを選択しない場合は、ベースラインとして以下のデフォルト値（このラボで使ってきた値）が設定されています:

```
--learning_rate 0.01
--decay_rate 1.0
--decay_steps 1000
--num_epochs 1
```

次のセルで、これらの値を変更して実行してみましょう。前よりも精度が上がるかどうか確かめてください。
ラボの時間制限もあるので100epoch以上の学習を行うことはお勧めしませんが、実用ではそれ以上のepoch数で学習する可能性が高いです。

都合の良いことに、学習を開始してからepoch数が大きすぎたということに気がついた場合は、途中で学習を止めることができ、そのモデルでテスト（2つ下のセルで実行）を行うことができます。TensorFlowにはチェックポイントの機能があり、モデルを定期的に保存することができるので、学習を途中で中断したとしても、最新のモデルが保持されているというわけです。

In [ ]:
!python exercises/cnnDice/runTraining.py --data_dir /data --num_epochs 1 --learning_rate 0.01 --decay_rate 1.0 --decay_steps 1000

In [ ]:
!python exercises/cnnDice/runEval.py --data_dir /data

1つの良い例では、86%の精度が得られました。
解答例の[A](#A)を参照して、ここで使ったパラメータを確認してみてください。 

# さらなる強化 

このラボでは説明のために、時間内に実行できる小さなタスクに焦点を当ててきました。しかし実際に各応用場面で画像セグメンテーションを遂行する際には、何を行うべきでしょうか。私たちがやるべきことは次の通りです。


* さらに長く学習する -- ここでは非常に短時間の学習しか行いませんでしたが、実際はもっと多くのepoch数で実行します。
* 学習データを増やす -- 私たちの学習データセットの中には、画像が236枚しかありませんでした。データを集め、さらにデータオーグメンテーションを行うこともできます。TensorFlowには、画像を自動的に反転/回転/転置するための組み込み関数があります。
* ネットワークをより大規模にする -- AlexNetやその他の大規模なネットワークをFCNに変換することができます。

## まとめ

このラボでは、一般に好まれているフレームワークであるTensorFlowを用いて画像セグメンテーションに取り組む機会を提供しました。
あなたはスタンダードなCNNを、セグメンテーションネットワークとして用いられるFCNに書き換える方法を学びました。
さらに、ネットワークの学習では適切な精度指標の選択がいかに大切であるかを学びました。
最後に、ディープラーニングのワークフローの一部としてパラメータ探索を行うことが、手元のタスクで許容できる精度を得るうえで重要であることを学びました。

## より多くのことを学ぶために

興味のある方は、以下を参照してください。

* [CUDA Developer Zone](https://developer.nvidia.com/category/zone/cuda-zone)で、より多くのことを学ぶことができます。
* NVIDIAのGPUをお持ちの方は、[CUDA tookit](https://developer.nvidia.com/cuda-toolkit)をインストールして利用できます。
* 無料オンライン講座Udacityでは、CUDA Cを用いた[Intro to Parallel Programming](https://www.udacity.com/course/cs344)というコースが受講できます。
* [Stackoverflow](http://stackoverflow.com/questions/tagged/cuda)でcudaタグを利用し、検索したり質問したりしてみましょう。

<a id="post-lab"></a>
## ラボを終了する前に

インスタンスを閉じる前に、あなたの行った作業をローカルに保存しておきましょう！

1. メニューバーの `File -> Download as -> IPython (.ipynb)` で、このIPython Notebookを保存することができます。

<a id="FAQ"></a>
---
# よくある質問

Q: セルが上手く実行されないのはなぜでしょうか。<br>
A: [こちら](https://developer.nvidia.com/self-paced-labs-faq#Troubleshooting)のFAQをご覧ください。

Q: タスクの実行中に、予期しない動作（誤った出力）が発生してしまいます。<br>
A: CUDA Runtime APIがエラーを返している可能性があります。CUDAランタイムエラーに関するメッセージが表示されていませんか？

<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

<a id="References"></a>
# 参考文献

<a id="1"></a>
[1] Sunnybrook cardiac images from earlier competition http://smial.sri.utoronto.ca/LV_Challenge/Data.html

<a id="2"></a>
[2] This "Sunnybrook Cardiac MR Database" is made available under the CC0 1.0 Universal license described above, and with more detail here: http://creativecommons.org/publicdomain/zero/1.0/

<a id="3"></a>
[3] Attribution:
Radau P, Lu Y, Connelly K, Paul G, Dick AJ, Wright GA. "Evaluation Framework for Algorithms Segmenting Short Axis Cardiac MRI." The MIDAS Journal -Cardiac MR Left Ventricle Segmentation Challenge, http://hdl.handle.net/10380/3070

<a id="4"></a>
[4] http://fcn.berkeleyvision.org/

<a id="5"></a>
[5] Long, Shelhamer, Darrell; "Fully Convoutional Networks for Semantic Segmentation", CVPR 2015.

<a id="6"></a>
[6] Zeiler, Krishnan, Taylor, Fergus; "Deconvolutional Networks", CVPR 2010.

<a id="7"></a>
[7] https://www.kaggle.com/c/second-annual-data-science-bowl/details/deep-learning-tutorial


# Solutions

<a id="A"></a>
[A] 以下の設定で、86%の精度が達成できます。

```
--learning_rate 0.03
--decay_rate 0.75
--num_epochs 100
--decay_steps 10000
OUTPUT: 2017-01-27 20:19:08.702868: Dice metric = 0.862
```